## 5. Local Avoidance

The goal of this section is to allow the robot, in case an obstacle is not detected by the webcam and image processing section of the code, to avoid this "invisible" obstacle and to succesfully return on the optimal path that was computed with A star once there are no more obstacle detected. Our local avoidance was done using a form of ANN (artificial neural network).

In our main function, an if-else statement first check if a path has indeed been made

```python
if path is None:
    print('No path found')
	continue
else:
    ... #rest of the code
```

Once it has been verified that a path as indeed been made, the program will check if we are walking down the path or trying to go back to it using an other if-else statement

```python
if correction_start:
    ... #rest of the code if we are corresting the position
else:
    ... #rest of the code if we are not and where local avoidance is
```

Another if statement will then check if we are far enough from the goal, because there is no point in making a local avoidance of obstacle if we are already on the goal. We can in the code (given below) that the variable u is use to store the value that is use for our PD controller which is the following : $$P * error + D * (error - prev\_error)$$

The local neural network function (local_nn) is then called.

```python

#Checks if we are not already on the goal
if euclidean_distance(state[:2], goal_c) > 30 and iter >=0:
    if projected_position(path, state[:2], iter) < 25 or euclidean_distance(state[:2], path[iter]) < 30:
        iter -= 1
    u = controller.control(state, path[iter])
    u = local_nn(th, u[0], u[1])
    predicted_state, P = filter.predict(state, P, u)

    set_motors(th, u[0], u[1])

#If we are on the goal, we stop
else:
    set_motors(th)
```

The local_nn function takes as argument th which is the communication with the Thymio as well as v and w that are, as we can see in the main, the different components of the u variable.

Because it is based an artificial neural network, we first declare weights for the different horizontal sensors that will impact the speed of the left and right wheel.

So, in the function, wl is the weight of the sensors for the left wheel and wr is the weight of the sensors for the right wheel.
The given values were find to be the most optimal one for the used Thymio.

Than the speed of the left and right motors are calculated by multplying the different weights to the sensors values and dividing it by a scaling factor.

Finally, we substract the left speed to the right one because if both sides have a change in speed, the result change for the robot is the difference between the two.

The function then returns the v and w that will be stored in u (in the main) and will be used to update the motors speed.

```python
def local_nn(th, v, w):
    
    
	wl = [1, 2, 3, -2, -1, 1, -1]
	wr = [-1, -2, -3, 2, 1, -1, 1]
	scale = 40
	L = 95 #mm
	speedL = np.dot(th["prox.horizontal"], wl)//scale
	speedR = np.dot(th["prox.horizontal"], wr)//scale

	w = w + (speedR - speedL)/(2*L)

	return v, w
```